# Assignment 3:  Decision Tree Implementation
*Margaret Thomann - February 17, 2018 *

In this assignment, I will construct a decision tree from the data provided about heart disease.

### Reading the data and assigning counts to arrays and Data class

#### Data Class 
A Data class will be instantiated for each line of the data.  It will then be added to one of two arrays (explained later).

In [301]:
from collections import OrderedDict

class Data:
    def __init__(self, class_value):
        self.class_value = class_value
        self.data_vars = OrderedDict()

### Calculate Information Gain for Each Feature
The below function can be used to determine the information gain for a given data and hypothesis (passed in as a string - x and y).  Information Gain can be represented as: Infgain(Y|X_K) = H(Y) - H(Y|X_K)

In [302]:
# Arrays for the Data instances
#     absence_heart_array  : contains all Data instantiations where heart disease is absent
#     presence_heart_array : contains all Data instantiations where heart disease is absent
absence_heart_array = []
presence_heart_array = []
total_data_array = []

# Classify the features according to their type: nominal or continuous
indices_for_nominal = [10, 1, 5, 8, 6, 2, 12]
indices_for_continuous = [0,3,4,7,9,11]
feature_names = ["age", "sex", "chest_pain_type", "resting_blood_pressure", "serum_cholesterol", "fasting_blood_sugar",
                "resting_electrocardiographic_results", "maximum_heart_rate_achieved", "exercise_induced_angina",
                "oldpeak", "slope_peak_exercise", "number_of_major_vessels", "thal", "has_heart_disease"]
features_and_types = OrderedDict()
for feature in feature_names:
    if feature_names.index(feature) in indices_for_continuous:
        features_and_types[feature] = "continuous"
    else:
        features_and_types[feature] = "nominal"

# Process the data and store it in the arrays
data = open('heart.data.txt')
for line in data.readlines():
    feature_value_list = line.split()
    has_heart_disease = int(feature_value_list[-1])
    data = Data(has_heart_disease)
    counter = 0
    feature_dict = OrderedDict()
    for feature in feature_value_list:
        data.data_vars[feature_names[counter]] = float(feature)
        counter += 1
    if has_heart_disease == 2:
        presence_heart_array.append(data)
    elif has_heart_disease == 1:
        absence_heart_array.append(data)
    total_data_array.append(data)

presence_heart_array_num = len(presence_heart_array)
absence_heart_array_num = len(absence_heart_array)
print "✔ Data processed"
print "-----------------"
print "\t" + str(presence_heart_array_num) + " = # Of People with Heart Disease"
print "\t" + str(absence_heart_array_num) + " = # Of People without Heart Disease"

✔ Data processed
-----------------
	120 = # Of People with Heart Disease
	150 = # Of People without Heart Disease


In [303]:
import math

def compute_info_gain(y, x, buckets):
    
    # Define dicts for the counts
    positive_y_counts = {}
    positive_x_counts = {}
    negative_y_counts = {}
    negative_x_counts = {}
    
    # Get the bucket values
    for bucket in buckets:
        # Convert to string
        s = ""
        for num in list(set(bucket)):
            s += (str(num)+ " ")
        positive_x_counts[s] = 0
        negative_x_counts[s] = 0        

    
    y_denom = 0
    for data in presence_heart_array:
        y_denom += 1
        value = data.data_vars[y]
        
        # Value is not in dictionary yet
        # so set the occurence for that value to 1
        if value not in positive_y_counts.keys():
            positive_y_counts[value] = 1 
        # Value is already in dictionary
        # so increase the occurence count for that value by 1
        else:
            current_count_for_value = positive_y_counts[value]
            positive_y_counts.update({value:current_count_for_value+1})
            
        # Same thing is done for processing x:
        x_value = data.data_vars[x]
        for key in negative_x_counts.keys():
            if str(x_value)+" " in key:
                current_count_for_value = positive_x_counts[key]
                positive_x_counts.update({key:current_count_for_value+1})
            
    
    for data in absence_heart_array:
        y_denom += 1
        value = data.data_vars[y]
        
        # Value is not in dictionary yet
        # so set the occurence for that value to 1
        if value not in negative_y_counts.keys():
            negative_y_counts[value] = 1 
        # Value is already in dictionary
        # so increase the occurence count for that value by 1
        else:
            current_count_for_value = negative_y_counts[value]
            negative_y_counts.update({value:current_count_for_value+1})
            
        # Same thing is done for processing x:
        x_value = data.data_vars[x]
        for key in negative_x_counts.keys():
            if str(x_value)+" " in key:
                current_count_for_value = negative_x_counts[key]
                negative_x_counts.update({key:current_count_for_value+1})
            
            
    # Calculate H(Y)     
    h_of_y = 0
    for count in positive_y_counts.values():
        p = float(float(count+1)/float(y_denom))
        entropy = -1 * p * (math.log(p, 2))
        h_of_y += entropy
    for count in negative_y_counts.values():
        p = float(float(count+1)/float(y_denom))
        entropy = -1 * p * (math.log(p, 2))
        h_of_y += entropy
    
    h_of_y_given_x = 0
    for feature_value in positive_x_counts.keys():
        #print "positive_x_counts: "+str(positive_x_counts[feature_value])+" for feature value: "+str(feature_value)
        #print "negative_x_counts: "+str(negative_x_counts[feature_value])+" for feature value: "+str(feature_value)
        
        sum_of_values = positive_x_counts[feature_value] + negative_x_counts[feature_value]
        #print "sum of values: "+str(sum_of_values)
        fraction = float(float(sum_of_values)/float(y_denom))
        p_positive = float(float(positive_x_counts[feature_value]+1)/float(sum_of_values)) 
        p_negative = float(float(negative_x_counts[feature_value]+1)/float(sum_of_values)) 
        entropy_positive = -p_positive * (math.log(p_positive, 2))
        entropy_negative = -p_negative * (math.log(p_negative, 2))
        h_of_y_given_x -= fraction*(entropy_positive+entropy_negative)
    
    info_gain = h_of_y - (-1*h_of_y_given_x)
    return info_gain
    

### Utility Function used to generate all of the possible splits
Credit:  https://stackoverflow.com/questions/19368375/set-partitions-in-python/30134039#30134039

In [304]:
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller

### Determine possible splits
These will be used for the information gain

In [305]:
# get_continuous_binary_split(feature):     
#    Gets a feature and splits the data for that feature in all the possible ways to split that
#    data into two buckets.  It returns a dictionary where the key is the number it split on and the
#    value is a list of two lists.  The first element of the list is all of the elements less than
#    the split and the second element of the list is a list of all of the elements greater than or
#    equal to the split.
#    For example:
#           feature = "age"
#           splits_dict = {50: [[20,40,43...],[50,60,61...]], 60: [[57,45,59...],[60,61,63...]]}
def get_continuous_binary_split(feature):
    # Ensure the function is being called only with continuous features
    if features_and_types[feature] != "continuous":
        raise ValueError('Error in get_continuous_binary_split: input feature is not continuous.')
        return
    
    # Create a list of the possible splits
    splits = []
    for data in total_data_array:
        feature_value = float(data.data_vars[feature])
        if feature_value not in splits:
            splits.append(feature_value)
    
    # Process the splits and create the less than list and greater than or equal to list for each
    splits_list = []
    for split in splits:
        lt_split_feature_values = []
        gtequal_split_feature_values = []
        for data in total_data_array:
            feature_value = float(data.data_vars[feature])
            if feature_value < split:
                lt_split_feature_values.append(feature_value)
            else:
                gtequal_split_feature_values.append(feature_value)
        if len(lt_split_feature_values) != 0:
            splits_list.append([lt_split_feature_values, gtequal_split_feature_values])
    
    return splits_list

In [306]:
import itertools
# get_nominal_split(feature):     
#    Gets a nominal feature and splits the data for that feature in all the possible ways to split that
#    data into every possible number of buckets.  It returns a dictionary where the key is the number it 
#    split on and the value is a list of the lists it generated from the split.  

def split_list(data, n):
    from itertools import combinations, chain
    for splits in combinations(range(1, len(data)), n-1):
        result = []
        prev = None
        for split in chain(splits, [None]):
            result.append(data[prev:split])
            prev = split
        yield result
        
def get_nominal_split(feature):
    # Ensure the function is being called only with nominal features
    if features_and_types[feature] != "nominal":
        raise ValueError('Error in get_nominal_split: input feature is not nominal.')
        return
    
    # Determine the unique values for the feature and the counts for the feature_value
    splits = {}
    split_nums = []
    for data in total_data_array:
        feature_value = float(data.data_vars[feature])
        if feature_value not in splits:
            split_nums.append(feature_value)
            splits[feature_value] = 1
        else:
            current_num = splits[feature_value]
            splits[feature_value] = current_num+1

    # Generate all the possible ways to partition the numbers  
    # possible_partitions is a list of tuples where the first element of the tuple
    # is the kth possible partition and the second element of the tuple is a list
    # of lists of partitions
    possible_partitions = []
    for p in partition(split_nums):
        # Be sure to remove the split where all values are placed in one bucket
        if len(sorted(p)) != 1:
            possible_partitions.append(sorted(p))
                
    # Generate a list of partition dicts
    # Each list element will be a list of dictionaries
    # The dictionaries will contain the feature_value and the number of times it occurs
    # The list of these dictionaries will be split up by partition
    partitions_and_values = []
    for split_lists in possible_partitions:
        split_dicts = []
        for bucket in split_lists:
            bucket_dict = {}
            for feature_value in bucket:
                num_of_feature_value_occurences = splits[feature_value]
                bucket_dict[feature_value] = num_of_feature_value_occurences
            split_dicts.append(bucket_dict)
        partitions_and_values.append(split_dicts)
    return partitions_and_values

In [307]:
def compute_best_split(y, feature):
    # Check if the feature is nominal or continuous and split accordingly
    splits = []
    if features_and_types[feature] == "nominal":
        splits = get_nominal_split(feature)
    else:
        splits = get_continuous_binary_split(feature)
    
    # Create a dictionary where the key is the information gain from a particular
    # split and the value is that particular split
    info_gains = {}
    for split in splits:
        info_gain = compute_info_gain(y, feature, split)
        info_gains[info_gain] = split
        
    # Process the dictionary and determine the highest info gain
    max_info_gain = max(info_gains, key=float)
    split_yielding_max_info_gain = info_gains[max_info_gain]
    
    # Print the maximum info gain and corresponding split
    print "FEATURE: "+feature
    print "Greatest Info Gain is: "+str(max_info_gain)
    print "from split:"
    for split in split_yielding_max_info_gain:
        print sorted(set(split))
    return max_info_gain

### Recurse to build the tree
Used the information gain function to determine the best splits for each node of the tree.

In [308]:
def build_tree():
    info_gains = {}
    for feature in data.data_vars.keys():
        y = "has_heart_disease"
        if feature != y:
            info_gain = compute_best_split(y, feature)
            info_gains[feature] = info_gain
    
    for feature in info_gains.keys():
        
        print str(info_gains[feature])+"\t"+feature


In [309]:
build_tree()

FEATURE: age
Greatest Info Gain is: 0.059167521759
from split:
[29.0, 34.0, 35.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0]
[55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 74.0, 76.0, 77.0]
